In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%load_ext tensorboard

#Cargamos los datos a trabajar

x_train_pre=np.load("./data/train_signals.npy")
y_train_pre=np.load("./data/train_marks.npy")
trainLen=len(x_train_pre)

x_val_pre=np.load("./data/val_signals.npy")
y_val_pre=np.load("./data/val_marks.npy")
valLen=len(x_val_pre)

x_test_pre=np.load("./data/test_signals.npy")
y_test_pre=np.load("./data/test_marks.npy")
testLen=len(x_test_pre)

#Creamos una funcion que muestree a un ratio deseado

def sampling(signals,samples,lenght,rate):
    x_new=[]
    for i in signals:
        x=[]
        for j in range(0,lenght,rate):
            x.append(i[j])
        x_new.append(x)
    return np.array(x_new)

#Obtenemos los nuevos resultados de nuestras señales a trabajar

x_train=sampling(x_train_pre,trainLen,800,1)
x_val=sampling(x_val_pre,valLen,800,1)
x_test=sampling(x_test_pre,testLen,800,1)

#Creamos una funcion que clasifique con 1 o 0 los valores de la señal a trabajar

def oneHot(marks,lenght,rate):
    y_new=[]
    for i in marks:
        y=[]
        for j in range(0,lenght,rate):
            if(i[0]<=j & j<=i[1]):
                y.append(1)
            else:
                y.append(0)
        y_new.append(y)
    return np.array(y_new)

#Obtenemos los nuevos resultados de nuestras marcas a trabajar

y_train=oneHot(y_train_pre,800,1)
y_val=oneHot(y_val_pre,800,1)
y_test=oneHot(y_test_pre,800,1)

x_train2=np.expand_dims(x_train,2)
y_train2=np.expand_dims(y_train,2)

x_val2=np.expand_dims(x_val,2)
y_val2=np.expand_dims(y_val,2)

x_test2=np.expand_dims(x_test,2)
y_test2=np.expand_dims(y_test,2)

BATCH_SIZE=20
repeat=20

training_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train2, y_train)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices(
    (x_test2, y_test)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (x_val2, y_val)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

time=np.arange(0,800,1)

In [9]:
y_train.shape

(4463, 800)

In [15]:
#Modelo Paper
prob=0.5
model = tf.keras.Sequential()

for i in range(0,5):
    model.add(tf.keras.layers.Conv1D(40, 7, activation='elu',padding="same"))
    model.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=1,padding="same"))   

model.add(tf.keras.layers.Conv1D(1, 1))


    

model.add(tf.keras.layers.Activation(tf.nn.softmax))

model.build(input_shape=(None,800,1))

model.compile(loss='categorical_crossentropy',optimizer="adam",  metrics=[tf.keras.metrics.MeanIoU(num_classes=2),'accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 800, 40)           320       
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 800, 40)           0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 800, 40)           11240     
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 800, 40)           0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 800, 40)           11240     
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 800, 40)           0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 800, 40)          

In [16]:
#Modelo Propuesto

prob = 0.5

model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Conv1D(16, 7, activation='elu', input_shape = (800,1)))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(32, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(48, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(64, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Flatten())

model2.add(tf.keras.layers.Dense(1600, activation=tf.nn.relu, name='fc2'))
model2.add(tf.keras.layers.Dropout(0.5))

model2.add(tf.keras.layers.Dense(1200, activation=tf.nn.relu, name='fc3'))
model2.add(tf.keras.layers.Dropout(0.5))

model2.add(tf.keras.layers.Dense(800, activation=tf.nn.relu, name='fc4')) 

model2.add(tf.keras.layers.Softmax(axis = 1))


model2.build(input_shape=(None,800,1))

model2.compile(loss='categorical_crossentropy',
               optimizer="adam",
               metrics=[tf.keras.metrics.MeanIoU(num_classes=2),'accuracy'])

model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_48 (Conv1D)           (None, 794, 16)           128       
_________________________________________________________________
max_pooling1d_43 (MaxPooling (None, 395, 16)           0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 389, 32)           3616      
_________________________________________________________________
dropout_35 (Dropout)         (None, 389, 32)           0         
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, 193, 32)           0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 187, 48)           10800     
_________________________________________________________________
dropout_36 (Dropout)         (None, 187, 48)          

In [17]:
model.fit(
    training_dataset, 
    epochs=10,
    steps_per_epoch=20,
    validation_data=val_dataset,
    validation_freq=1,
    callbacks=[tf.keras.callbacks.TensorBoard('./logs/model1_finalfinal2', update_freq=100)]
)



Epoch 1/10
20/20 [==============================] - 7s 363ms/step - loss: 1116.3636 - mean_io_u_5: 0.1044 - accuracy: 0.2088 - val_loss: 1177.6301 - val_mean_io_u_5: 0.1101 - val_accuracy: 0.2202
Epoch 2/10
20/20 [==============================] - 5s 253ms/step - loss: 1165.6958 - mean_io_u_5: 0.1090 - accuracy: 0.2180 - val_loss: 1177.6304 - val_mean_io_u_5: 0.1101 - val_accuracy: 0.2202
Epoch 3/10
20/20 [==============================] - 5s 252ms/step - loss: 1149.5023 - mean_io_u_5: 0.1075 - accuracy: 0.2150 - val_loss: 1177.6304 - val_mean_io_u_5: 0.1101 - val_accuracy: 0.2202
Epoch 4/10
20/20 [==============================] - 5s 253ms/step - loss: 1157.2899 - mean_io_u_5: 0.1082 - accuracy: 0.2164 - val_loss: 1177.6298 - val_mean_io_u_5: 0.1101 - val_accuracy: 0.2202
Epoch 5/10
20/20 [==============================] - 5s 249ms/step - loss: 1155.4851 - mean_io_u_5: 0.1080 - accuracy: 0.2161 - val_loss: 1177.6302 - val_mean_io_u_5: 0.1101 - val_accuracy: 0.2202
Epoch 6/10
20/20 [==

In [18]:
model2.fit(
    training_dataset, 
    epochs=10,
    steps_per_epoch=20,
    validation_data=val_dataset,
    validation_freq=1,
    callbacks=[tf.keras.callbacks.TensorBoard('./logs/model2_finalfinal2', update_freq=100)]
)

Epoch 1/10
20/20 [==============================] - 8s 395ms/step - loss: 53160784.0000 - mean_io_u_6: 0.3948 - accuracy: 0.0000e+00 - val_loss: 49377852.0000 - val_mean_io_u_6: 0.3913 - val_accuracy: 0.0022
Epoch 2/10
20/20 [==============================] - 4s 176ms/step - loss: 9799462912.0000 - mean_io_u_6: 0.3921 - accuracy: 0.0000e+00 - val_loss: 5130813952.0000 - val_mean_io_u_6: 0.3911 - val_accuracy: 0.0000e+00
Epoch 3/10
20/20 [==============================] - 3s 172ms/step - loss: 416176209920.0000 - mean_io_u_6: 0.3927 - accuracy: 0.0025 - val_loss: 150598352896.0000 - val_mean_io_u_6: 0.3914 - val_accuracy: 0.0039
Epoch 4/10
20/20 [==============================] - 3s 163ms/step - loss: 6625682259968.0000 - mean_io_u_6: 0.3953 - accuracy: 0.0025 - val_loss: 2007057039360.0000 - val_mean_io_u_6: 0.3913 - val_accuracy: 0.0033
Epoch 5/10
20/20 [==============================] - 3s 160ms/step - loss: 60602621886464.0000 - mean_io_u_6: 0.3945 - accuracy: 0.0000e+00 - val_loss:

In [22]:

%tensorboard --logdir=.


Reusing TensorBoard on port 6006 (pid 6212), started 4 days, 0:05:56 ago. (Use '!kill 6212' to kill it.)

"kill" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [1]:
a=0.4
b=0.6


In [2]:
round(a)

0

In [3]:
round(b)

1